In [13]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [14]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
    
    for c in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter( 
            x=df_plot.time,
            y=df_plot[c],
            line=dict(width=2,color="#4290f5"),
            line_shape='spline',
            name=c,
            mode='lines'
            ))        

    df_markers = df_plot[df_plot.BB_SIGNAL!=0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor="#3480eb",
        increasing_line_color='#3480eb',  
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [15]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [16]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [17]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [18]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [19]:
df_plot = df.iloc[1000:1150]
plot_candles(df_plot)

In [20]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)

In [21]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
25,2020-01-08 02:00:00+00:00,108.030,108.461,108.007,108.305,108.257667,108.223183,0.187500,108.598184,107.848183,0,108.311917,108.318809,0.006892,0.006892,0.000000
26,2020-01-08 06:00:00+00:00,108.308,108.489,108.276,108.449,108.404667,108.226717,0.190343,108.607402,107.846031,0,108.323524,108.339061,0.015537,0.011695,0.003842
27,2020-01-08 10:00:00+00:00,108.446,108.816,108.424,108.670,108.636667,108.252400,0.209320,108.671041,107.833759,0,108.352554,108.390453,0.037899,0.022434,0.015465
28,2020-01-08 14:00:00+00:00,108.673,109.134,108.616,109.096,108.948667,108.297567,0.254806,108.807179,107.787955,-1,108.414245,108.499860,0.085614,0.043837,0.041778
29,2020-01-08 18:00:00+00:00,109.099,109.246,109.000,109.134,109.126667,108.348550,0.310577,108.969704,107.727396,-1,108.473443,108.598074,0.124630,0.067871,0.056759


In [24]:
df_plot = df.iloc[1000:1200]
fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2,color="#03adfc"),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))        
fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2,color="#fc4a03"),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))
fig.add_trace(go.Bar( 
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()